In [1]:
import numpy as np
import pandas as pd

# Creating a fake dataset
np.random.seed(42)  # For reproducibility

# Parameters for the fake dataset
num_samples = 200  # Number of samples
num_cues = 16      # Number of cues (features)
num_emotions = 8   # Number of emotions

# Generate random feature values for X (values between 0 and 1)
X = pd.DataFrame(np.random.rand(num_samples, num_cues), 
                      columns=[f'Cue {i+1}' for i in range(num_cues)])

# Generate random emotion labels (values between 0 and 7, as there are 8 emotions)
y = pd.Series(np.random.randint(0, num_emotions, num_samples), name='Emotion')


In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

# Parameters
num_emotions = 8
num_cues = 16
num_permutations = 1000
alpha_values = np.logspace(-6, 6, 13)

# Storing the results
observed_accuracies = np.zeros((num_emotions, num_cues))
p_values = np.zeros((num_emotions, num_cues))

# Perform Ridge Regression for each emotion and each cue
for emotion in range(num_emotions):
    y_emotion = (y == emotion).astype(int)  # Binary labels for current emotion
    
    for cue in range(num_cues):
        X_cue = X.iloc[:, cue].values.reshape(-1, 1)  # Extract cue as a single feature
        
        # Ridge regression with cross-validation
        model = RidgeCV(alphas=alpha_values, cv=5)
        observed_accuracy = cross_val_score(model, X_cue, y_emotion, cv=5, scoring='accuracy').mean()
        observed_accuracies[emotion, cue] = observed_accuracy
        
        # Permutation test
        permuted_accuracies = []
        for _ in range(num_permutations):
            y_permuted = shuffle(y_emotion)
            perm_accuracy = cross_val_score(model, X_cue, y_permuted, cv=5, scoring='accuracy').mean()
            permuted_accuracies.append(perm_accuracy)
        
        # Calculate p-value
        permuted_accuracies = np.array(permuted_accuracies)
        p_value = np.mean(permuted_accuracies >= observed_accuracy)
        p_values[emotion, cue] = p_value

# Apply multiple comparison correction if needed (e.g., Bonferroni correction)
adjusted_p_values = p_values * num_cues
adjusted_p_values = np.minimum(adjusted_p_values, 1.0)  # Ensure p-values are <= 1


results = pd.DataFrame(adjusted_p_values, columns=[f'Cue {i+1}' for i in range(num_cues)], 
                       index=[f'Emotion {i+1}' for i in range(num_emotions)])
results
